## Inicializar la Dimensión Fecha
Importaciones

In [8]:
from datetime import date
import pandas as pd
import numpy as np
import holidays
from sqlalchemy import create_engine
import yaml

# Añadir los atributos básicos como año, mes, día, trimestre, y semana del año.

In [9]:
# Crear un rango de fechas por minuto para obtener horas y minutos también
dates = pd.date_range(start='1/1/2023', end='12/31/2025 23:59', freq='D')

# Inicializar el DataFrame
dim_fecha = pd.DataFrame({
    "fecha_completa": dates,  # La fecha completa en formato YYYY-MM-DD HH:MM
})

dim_fecha.head()

,fecha_completa
0,2023-01-01
1,2023-01-02
2,2023-01-03
3,2023-01-04
4,2023-01-05


# Añadir columnas para horas y minutos, y marcar si es día laboral, fin de semana o feriado.

In [10]:
# Añadir columnas como el año, mes, día, día de la semana y el trimestre del año.

dim_fecha["fecha_id"] = dim_fecha["fecha_completa"].dt.strftime('%Y%m%d').astype(int)  # ID de fecha en formato YYYYMMDD
dim_fecha["año"] = dim_fecha["fecha_completa"].dt.year
dim_fecha["mes"] = dim_fecha["fecha_completa"].dt.month
dim_fecha["nombre_mes"] = dim_fecha["fecha_completa"].dt.month_name()
dim_fecha["semana"] = dim_fecha["fecha_completa"].dt.isocalendar().week
dim_fecha["dia"] = dim_fecha["fecha_completa"].dt.day
dim_fecha["nombre_dia"] = dim_fecha["fecha_completa"].dt.day_name()
dim_fecha["trimestre"] = dim_fecha["fecha_completa"].dt.quarter

dim_fecha.head()


,fecha_completa,fecha_id,año,mes,nombre_mes,semana,dia,nombre_dia,trimestre
0,2023-01-01,20230101,2023,1,January,52,1,Sunday,1
1,2023-01-02,20230102,2023,1,January,1,2,Monday,1
2,2023-01-03,20230103,2023,1,January,1,3,Tuesday,1
3,2023-01-04,20230104,2023,1,January,1,4,Wednesday,1
4,2023-01-05,20230105,2023,1,January,1,5,Thursday,1


# Añadir información sobre feriados y si es fin de semana.


In [11]:

co_holidays = holidays.CO(language="es")  # Usar librería holidays para identificar feriados en Colombia

# Aplicar las reglas para identificar si es feriado usando la columna correcta
dim_fecha["is_Holiday"] = dim_fecha["fecha_completa"].dt.date.apply(lambda x: x in co_holidays)
dim_fecha["holiday"] = dim_fecha["fecha_completa"].dt.date.apply(lambda x: co_holidays.get(x))

# Añadir fecha de guardado
dim_fecha["saved"] = date.today()

# Identificar si es fin de semana (sábado o domingo)
dim_fecha["weekend"] = dim_fecha["fecha_completa"].dt.weekday >= 5

# Mostrar las primeras filas del DataFrame para verificar
dim_fecha.head()


,fecha_completa,fecha_id,año,mes,nombre_mes,semana,dia,nombre_dia,trimestre,is_Holiday,holiday,saved,weekend
0,2023-01-01,20230101,2023,1,January,52,1,Sunday,1,True,Año Nuevo,2024-11-04,True
1,2023-01-02,20230102,2023,1,January,1,2,Monday,1,False,None,2024-11-04,False
2,2023-01-03,20230103,2023,1,January,1,3,Tuesday,1,False,None,2024-11-04,False
3,2023-01-04,20230104,2023,1,January,1,4,Wednesday,1,False,None,2024-11-04,False
4,2023-01-05,20230105,2023,1,January,1,5,Thursday,1,False,None,2024-11-04,False


# Añadir días laborales y fines de semana

In [12]:
# Añadir indicadores booleanos para días laborales y fines de semana
dim_fecha["es_dia_laboral"] = dim_fecha["fecha_completa"].dt.weekday < 5  # True si es de lunes a viernes
dim_fecha["es_fin_de_semana"] = dim_fecha["fecha_completa"].dt.weekday >= 5  # True si es sábado o domingo

# Marcar feriados en Colombia
co_holidays = holidays.CO(years=dim_fecha["año"].unique(), language="es")
dim_fecha["es_feriado"] = dim_fecha["fecha_completa"].dt.date.isin(co_holidays)

dim_fecha.head()

,fecha_completa,fecha_id,año,mes,nombre_mes,semana,dia,nombre_dia,trimestre,is_Holiday,holiday,saved,weekend,es_dia_laboral,es_fin_de_semana,es_feriado
0,2023-01-01,20230101,2023,1,January,52,1,Sunday,1,True,Año Nuevo,2024-11-04,True,False,True,True
1,2023-01-02,20230102,2023,1,January,1,2,Monday,1,False,None,2024-11-04,False,True,False,False
2,2023-01-03,20230103,2023,1,January,1,3,Tuesday,1,False,None,2024-11-04,False,True,False,False
3,2023-01-04,20230104,2023,1,January,1,4,Wednesday,1,False,None,2024-11-04,False,True,False,False
4,2023-01-05,20230105,2023,1,January,1,5,Thursday,1,False,None,2024-11-04,False,True,False,False


# Conexión a la Base de Datos

In [13]:
# Cargar configuración de conexión desde el archivo YAML
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)


# Cargar la Dimensión Fecha a la Base de Datos
Cargar el DataFrame `dim_fecha` a la tabla de la base de datos usando SQLAlchemy.

In [14]:
dim_fecha.to_sql('dim_fecha', etl_conn, if_exists='replace', index=False)

96